In [ ]:
!pip install torch_geometric
!pip install rdkit-pypi

import torch
import torch_geometric
import numpy as np
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
import pickle

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU instead")

from google.colab import drive

drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 21.5 MB/s eta 0:00:00
GPU is available
Mounted at /content/drive


In [ ]:
from torch_geometric.data import DataLoader

batch_size=32
test_dataset = torch.load("/content/drive/MyDrive/2023 Fall/cosi149 - project 3/project 3/test_data.pt/test_data.pt")
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
# Atom encoder

class AtomEncoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(AtomEncoder, self).__init__()

        self.embeddings = torch.nn.ModuleList()

        for i in range(9):
            self.embeddings.append(torch.nn.Embedding(100, hidden_channels))

    def reset_parameters(self):
        for embedding in self.embeddings:
            embedding.reset_parameters()

    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(1)

        out = 0
        for i in range(x.size(1)):
            out += self.embeddings[i](x[:, i])

        return out

In [ ]:
from torch_geometric.nn import AttentiveFP

loaded_model = AttentiveFP(64,128,12,3,3,3, 0.5).to(device)
emb = AtomEncoder(64)

In [ ]:
loaded_model.load_state_dict(torch.load('/content/drive/MyDrive/attentiveFP_1.pt',map_location='cuda'))

<All keys matched successfully>

In [ ]:
loaded_model.eval()
list = []
for batch in test_loader:

        batch = batch.to(device)
        if batch.x.shape[0] == 1:
            pass
        else:
            with torch.no_grad():
                pred = loaded_model(emb(batch.x.to('cpu')).to(device), batch.edge_index.to(device), batch.edge_attr.to(device), batch.batch.to(device))
            for tensor in pred:
                array = tensor.cpu().numpy().tolist()
                list.append(array)





In [ ]:
list

[[-3.489567995071411,
  -3.7453854084014893,
  -2.0889830589294434,
  -1.676667332649231,
  -1.5529141426086426,
  -2.2954609394073486,
  -2.995102882385254,
  -0.6125901341438293,
  -2.8992233276367188,
  -3.0398800373077393,
  -0.1825263798236847,
  -1.684305191040039],
 [-3.857630729675293,
  -4.455156326293945,
  -2.995307445526123,
  -3.956843137741089,
  -2.529968500137329,
  -3.9727823734283447,
  -4.434045791625977,
  -2.2833118438720703,
  -4.08447265625,
  -3.774832248687744,
  -3.012615442276001,
  -3.6875617504119873],
 [-4.084919452667236,
  -4.798502445220947,
  -3.1009886264801025,
  -4.33090877532959,
  -2.729158639907837,
  -4.358662128448486,
  -4.749710559844971,
  -2.551591396331787,
  -4.287747859954834,
  -3.9941978454589844,
  -3.4164950847625732,
  -4.041238784790039],
 [-3.644512891769409,
  -4.862853527069092,
  -3.5248241424560547,
  -3.2970316410064697,
  -3.0728960037231445,
  -4.673333168029785,
  -5.3845672607421875,
  -2.6338958740234375,
  -4.9905200004

In [ ]:
import csv

output_file = "/content/drive/MyDrive/test_output.csv"

with open(output_file, "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    for prediction in list:
        csv_writer.writerow(prediction)